# [6.2 Forward Planning](http://artint.info/2e/html/ArtInt2e.Ch6.S2.html)
- [Implementation Details](http://artint.info/AIPython/aipython.pdf#page=85) (page 85)

## About

A forward planner searches the state-space graph from the initial state looking for a state that satisfies a goal description. It can use any of the search strategies we learnt before.

## Instructions

Each section header contains a link to the corresponding chapter in the accompanying textbook, and an "Implementation Details" link provided throughout tells you how the implementation works. Before using this notebook, make sure you have followed the [installation instructions](https://aispace2.github.io/AISpace2/install.html) beforehand.

You can run each cell by selecting it and pressing *Ctrl+Enter*. Alternatively, you can click the *Play* button in the toolbar, to the left of the stop button. 

For more information, including how the code in this notebook differs from that in [AIPython](aipython.org), check out the [Reference](https://aispace2.github.io/AISpace2/reference.html).

In [ ]:
from aipython.searchProblem import Arc, Search_problem
from aipython.stripsProblem import Strips, STRIPS_domain
from aipython.stripsForwardPlanner import State, zero


class Forward_STRIPS(Search_problem):
    """A search problem from a planning problem where:
    * a node is a state object.
    * the dynamics are specified by the STRIPS representation of actions
    """

    def __init__(self, planning_problem, heur=zero):
        """creates a forward seach space from a planning problem.
        heur(state,goal) is a heuristic function,
           an underestimate of the cost from state to goal, where
           both state and goals are feature:value dictionaries.
        """
        self.prob_domain = planning_problem.prob_domain
        self.initial_state = State(planning_problem.initial_state)
        self.goal = planning_problem.goal
        self.heur = heur

    def is_goal(self, state):
        """is True if node is a goal.

        Every goal feature has the same value in the state and the goal."""
        state_asst = state.assignment
        return all(prop in state_asst and state_asst[prop] == self.goal[prop]
                   for prop in self.goal)

    def start_node(self):
        """returns start node"""
        return self.initial_state

    def neighbors(self, state):
        """returns neighbors of state in this problem"""
        cost = 1
        state_asst = state.assignment
        return [Arc(state, self.effect(act, state_asst), cost, act)
                for act in self.prob_domain.actions
                if self.possible(act, state_asst)]

    def possible(self, act, state_asst):
        """True if act is possible in state.
        act is possible if all of its preconditions have the same value in the state"""
        preconds = self.prob_domain.strips_map[act].preconditions
        return all(pre in state_asst and state_asst[pre] == preconds[pre]
                   for pre in preconds)

    def effect(self, act, state_asst):
        """returns the state that is the effect of doing act given state_asst"""
        new_state_asst = self.prob_domain.strips_map[act].effects.copy()
        for prop in state_asst:
            if prop not in new_state_asst:
                new_state_asst[prop] = state_asst[prop]
        return State(new_state_asst)

    def heuristic(self, state):
        """in the forward planner a node is a state.
        the heuristic is an (under)estimate of the cost
        of going from the state to the top-level goal.
        """
        return self.heur(state.assignment, self.goal)

In [ ]:
from aipython.stripsProblem import simple_problem1, simple_problem2, blocks1, blocks2, blocks3
from aipython.searchMPP import SearcherMPP
from IPython.display import display

a = SearcherMPP(Forward_STRIPS(problem0))
a.show_edge_costs = False
a.search()
display(a)

**Note**: Run the method below to find _additional_ solutions. You should _not_ run this until you have already finished finding a solution because it will cause the state of the frontier to be indeterminate. You will know when you have found a solution when the output says "n paths have been expanded".

In [ ]:
a.search()

A heuristic function can also be used with the forward planner.

In [ ]:
from aipython.searchMPP import SearcherMPP
from aipython.stripsProblem import simple_problem1, simple_problem2, blocks1, blocks2, blocks3
from aipython.stripsHeuristic import heuristic_fun
from IPython.display import display

m = SearcherMPP(Forward_STRIPS(simple_problem1, heuristic_fun))
m.show_node_heuristics = True
m.search()
display(m)

**Note**: Run the method below to find _additional_ solutions. You should _not_ run this until you have already finished finding a solution because it will cause the state of the frontier to be indeterminate. You will know when you have found a solution when the output says "n paths have been expanded".

In [ ]:
m.search()